In [1]:
import pandas as pd
import warnings
import numpy as np
warnings.filterwarnings('ignore')
train = pd.read_csv('trains.csv')
test = pd.read_csv('test.csv')

for col in train.columns:
    col_type = train[col].dtypes
    min1 = train[col].min()
    max1 = train[col].max()
    if str(col_type)[:3] == 'int':
        train[col] = train[col].astype(np.int16)
    else:
        if min1 > np.finfo(np.float16).min and max1 < np.finfo(np.float16).max:
            train[col] = train[col].astype(np.float16)
        elif min1 > np.finfo(np.float32).min and max1 < np.finfo(np.float32).max:
            train[col] = train[col].astype(np.float32)
        else:
            train[col] = train[col].astype(np.float64)
train_X = train.iloc[:,4:]
train_Y = train.iloc[:,0:4]
test_X = test.iloc[:,1:]

In [2]:
np.random.seed(123)
i = np.random.choice(np.arange(0,810000), 10000, replace=False)
train_set = train_X.drop(i,axis=0) ,train_Y.drop(i,axis=0)
val_set = train_X.loc[i] ,train_Y.loc[i]

In [3]:
import keras
from keras.models import Sequential, Model
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import BatchNormalization
from keras import optimizers
from keras import activations
def build_model(lr, dropout_rate=0.5):
    uni = 160
    model = Sequential()
    model.add(Dense(uni, activation=swish, input_dim=226))
    model.add(BatchNormalization())
    model.add(Dense(uni, activation=swish, input_dim=226))
    model.add(BatchNormalization())
    model.add(Dropout(dropout_rate))
    model.add(Dense(uni, activation=swish, input_dim=226))
    model.add(BatchNormalization())
    model.add(Dense(uni, activation=swish, input_dim=226))
    model.add(BatchNormalization())
    model.add(Dense(uni, activation=swish, input_dim=226))
    model.add(BatchNormalization())
    model.add(Dropout(dropout_rate))
    model.add(Dense(uni, activation=swish, input_dim=226))
    model.add(BatchNormalization())
    model.add(Dense(uni, activation=swish, input_dim=226))
    model.add(BatchNormalization())
    model.add(Dense(uni, activation=swish, input_dim=226))
    model.add(Dense(uni, activation=swish, input_dim=226))
    model.add(Dense(4, activation='linear'))
    op = optimizers.Nadam(learning_rate=lr, beta_1 = 0.9, beta_2 = 0.999)
    model.compile(loss='mae', optimizer=op, metrics=['mae'])
    return model
def swish(x) :
    return x * keras.activations.sigmoid(x)

def mish(x) :
    return x * keras.activations.tanh(keras.activations.softplus(x)) 

Using TensorFlow backend.


In [8]:
def fit_with(verbose, lr, dropout_rate):
    model = build_model(lr, dropout_rate)
    fit_with = model.fit(train_set[0],train_set[1], epochs=15, batch_size=1000, shuffle=True)
    e_val = model.evaluate(val_set[0],val_set[1],steps=10, verbose=0)
    print('Test loss:', e_val[0])
    print('Test score:', e_val[1])
    return e_val[0]

In [9]:
from functools import partial

verbose = 1
fit_with_partial = partial(fit_with, verbose)

In [ ]:
from bayes_opt import BayesianOptimization

# Bounded region of parameter space
pbounds = {'dropout_rate': (0.0, 0.1), 'lr': (1e-4, 1e-2)}

bayes_optimizer = BayesianOptimization(
    f=fit_with_partial,
    pbounds=pbounds,
    verbose=2,  # verbose = 1 prints only when a maximum is observed, verbose = 0 is silent
    random_state=1,
)

bayes_optimizer.maximize(init_points=10, n_iter=10,)
for i, res in enumerate(bayes_optimizer.res):
    print("Iteration {}: \n\t{}".format(i, res))

print(bayes_optimizer.max)

|   iter    |  target   | dropou... |    lr     |
-------------------------------------------------
Epoch 1/15
800000/800000 [==============================] - 18s 23us/step - loss: 53.6197 - mae: 53.6197
Epoch 2/15
800000/800000 [==============================] - 16s 20us/step - loss: 27.2449 - mae: 27.2449
Epoch 3/15
800000/800000 [==============================] - 16s 20us/step - loss: 18.1218 - mae: 18.1218
Epoch 4/15
800000/800000 [==============================] - 16s 20us/step - loss: 15.0752 - mae: 15.0751
Epoch 5/15
800000/800000 [==============================] - 16s 20us/step - loss: 13.4544 - mae: 13.4544
Epoch 6/15
800000/800000 [==============================] - 16s 20us/step - loss: 12.5549 - mae: 12.5549
Epoch 7/15
800000/800000 [==============================] - 16s 21us/step - loss: 11.7580 - mae: 11.7580
Epoch 8/15
800000/800000 [==============================] - 17s 21us/step - loss: 11.2582 - mae: 11.2582
Epoch 9/15
800000/800000 [==============================] - 

Epoch 11/15
800000/800000 [==============================] - 16s 20us/step - loss: 9.1740 - mae: 9.1740
Epoch 12/15
800000/800000 [==============================] - 17s 21us/step - loss: 9.0138 - mae: 9.0138
Epoch 13/15
800000/800000 [==============================] - 16s 21us/step - loss: 8.7980 - mae: 8.7980
Epoch 14/15
800000/800000 [==============================] - 17s 21us/step - loss: 8.6158 - mae: 8.6158
Epoch 15/15
800000/800000 [==============================] - 16s 20us/step - loss: 8.4482 - mae: 8.4482
Test loss: 0.6098661899566651
Test score: 60.986618995666504
|  5        |  0.6099   |  0.1984   |  0.005434 |
Epoch 1/15
800000/800000 [==============================] - 18s 22us/step - loss: 50.0551 - mae: 50.0550
Epoch 2/15
800000/800000 [==============================] - 16s 21us/step - loss: 23.7387 - mae: 23.7387
Epoch 3/15
800000/800000 [==============================] - 17s 21us/step - loss: 16.9208 - mae: 16.9208
Epoch 4/15
800000/800000 [============================

800000/800000 [==============================] - 16s 21us/step - loss: 8.6913 - mae: 8.6913
Epoch 6/15
800000/800000 [==============================] - 17s 21us/step - loss: 7.9203 - mae: 7.9203
Epoch 7/15
800000/800000 [==============================] - 16s 21us/step - loss: 7.4526 - mae: 7.4526 1s - 
Epoch 8/15
800000/800000 [==============================] - 16s 21us/step - loss: 7.0293 - mae: 7.0293
Epoch 9/15
800000/800000 [==============================] - 17s 21us/step - loss: 6.7256 - mae: 6.7256
Epoch 10/15
800000/800000 [==============================] - 17s 21us/step - loss: 6.4903 - mae: 6.4903
Epoch 11/15
800000/800000 [==============================] - 17s 21us/step - loss: 6.2259 - mae: 6.2259
Epoch 12/15
800000/800000 [==============================] - 16s 20us/step - loss: 6.0463 - mae: 6.0463 1s - 
Epoch 13/15
800000/800000 [==============================] - 17s 21us/step - loss: 5.8779 - mae: 5.8779 2s - loss: 5.8862 - mae: 
Epoch 14/15
800000/800000 [===============

800000/800000 [==============================] - 17s 21us/step - loss: 12.2846 - mae: 12.2846 1s - loss - ETA: 0s - loss: 12.2836 - mae: 12.283
Epoch 13/15
800000/800000 [==============================] - 16s 20us/step - loss: 12.0486 - mae: 12.0486
Epoch 14/15
800000/800000 [==============================] - 17s 21us/step - loss: 11.8076 - mae: 11.8076
Epoch 15/15
800000/800000 [==============================] - 17s 21us/step - loss: 11.6397 - mae: 11.6397
Test loss: 0.7667818069458008
Test score: 76.67818021774292
|  14       |  0.7668   |  0.3136   |  0.01     |


In [10]:
model11 = build_model()
model11.fit(train_X, train_Y, epochs=100, batch_size=1000, validation_split = 0.05, shuffle=True)

Epoch 1/1


KeyboardInterrupt: 